In [1]:
!pip install https://github.com/elyase/geotext/archive/master.zip

  Using cached https://github.com/elyase/geotext/archive/master.zip
  Preparing metadata (setup.py) ... done


In [2]:
# Uninstall the current PyTorch installation
!pip uninstall -y torch torchvision torchaudio

# Install PyTorch with CUDA support. Replace 'cu118' with your CUDA version if different.
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.5.0+cu118
Uninstalling torch-2.5.0+cu118:
  Successfully uninstalled torch-2.5.0+cu118
Found existing installation: torchvision 0.20.0+cu118
Uninstalling torchvision-0.20.0+cu118:
  Successfully uninstalled torchvision-0.20.0+cu118
Found existing installation: torchaudio 2.5.0+cu118
Uninstalling torchaudio-2.5.0+cu118:
  Successfully uninstalled torchaudio-2.5.0+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.5.0%2Bcu118-cp310-cp310-linux_x86_64.whl (838.3 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.20.0%2Bcu118-cp310-cp310-linux_x86_64.whl (6.5 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.5.0%2Bcu118-cp310-cp310-linux_x86_64.whl (3.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2

In [3]:
# Install necessary libraries
!pip install transformers torch beautifulsoup4 requests gensim geopandas matplotlib

# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import requests
from bs4 import BeautifulSoup
from geotext import GeoText
import gensim
from gensim import corpora
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt

# Load NVIDIA's Llama model and tokenizer
model_name = "nvidia/Llama-3.1-Nemotron-70B-Instruct-HF"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize data structures to store news data
saved_news = []
summaries = []
spatial_data = []
temporal_data = []


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ImportError: /usr/local/lib/python3.10/dist-packages/_XLAC.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN2at6native12cpu_fallbackERKN3c1014OperatorHandleEPSt6vectorINS1_6IValueESaIS6_EEb

In [ ]:
def search_and_save_news():
    print("Searching for the latest news articles about Alzheimer’s disease...")

    # Multiple news URLs for Alzheimer's disease
    urls = [
        "https://www.alz.org/news",
        "https://www.mayoclinic.org/diseases-conditions/alzheimers-disease/news",
        "https://www.bbc.com/news/topics/c52ew7kze47t/alzheimers-disease",
        "https://www.nih.gov/news-events/news-releases/nih-funded-researchers-discover-new-insights-alzheimers-disease"
    ]

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                saved_news.append(url)
        except Exception as e:
            print(f"Error fetching {url}: {e}")

    print(f"Found {len(saved_news)} news URLs.")
    return saved_news

# Example usage
saved_news = search_and_save_news()
print(saved_news)


In [ ]:
def fetch_news_content(url):
    print(f"Fetching content from {url}...")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    paragraphs = soup.find_all('p')
    content = ' '.join([para.get_text() for para in paragraphs])
    return content[:2000]  # Limit content for this example

def call_nvidia_model(content):
    print("Summarizing content using NVIDIA Llama-3.1-Nemotron-70B...")

    # Prepare prompt for the model
    prompt = f"Summarize the following article: {content}"
    messages = [{"role": "user", "content": prompt}]

    # Tokenize the message for the NVIDIA model
    tokenized_message = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_dict=True)

    # Generate response using the model
    response_token_ids = model.generate(tokenized_message['input_ids'].cuda(),
                                        attention_mask=tokenized_message['attention_mask'].cuda(),
                                        max_new_tokens=512,
                                        pad_token_id=tokenizer.eos_token_id)

    # Decode the generated tokens to get the response
    generated_tokens = response_token_ids[:, len(tokenized_message['input_ids'][0]):]
    generated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return generated_text

def summarize_news():
    for url in saved_news:
        content = fetch_news_content(url)
        summary = call_nvidia_model(content)
        summaries.append(summary)
    print("News articles summarized successfully.")
    return summaries

# Example usage
summaries = summarize_news()
for i, summary in enumerate(summaries):
    print(f"Summary {i+1}:\n{summary}\n")


In [ ]:
def call_nvidia_model(content):
    print("Summarizing content using NVIDIA Llama-3.1-Nemotron-70B on GPU...")

    # Prepare prompt for the model
    prompt = f"Summarize the following article: {content}"
    messages = [{"role": "user", "content": prompt}]

    # Tokenize the message for the NVIDIA model
    tokenized_message = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_dict=True)

    # Generate response using the model on GPU
    response_token_ids = model.generate(tokenized_message['input_ids'].cuda(),
                                        attention_mask=tokenized_message['attention_mask'].cuda(),
                                        max_new_tokens=512,
                                        pad_token_id=tokenizer.eos_token_id)

    # Decode the generated tokens to get the response
    generated_tokens = response_token_ids[:, len(tokenized_message['input_ids'][0]):]
    generated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return generated_text


In [ ]:
def fetch_news_content(url):
    print(f"Fetching content from {url}...")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    paragraphs = soup.find_all('p')
    content = ' '.join([para.get_text() for para in paragraphs])
    return content[:2000]  # Limit content for this example

def call_nvidia_model(content):
    print("Summarizing content using NVIDIA Llama-3.1-Nemotron-70B on GPU...")

    # Prepare prompt for the model
    prompt = f"Summarize the following article: {content}"
    messages = [{"role": "user", "content": prompt}]

    # Tokenize the message for the NVIDIA model
    tokenized_message = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_dict=True)

    # Move input tensors to GPU (CUDA)
    input_ids = tokenized_message['input_ids'].cuda()
    attention_mask = tokenized_message['attention_mask'].cuda()

    # Generate response using the model on GPU
    response_token_ids = model.generate(input_ids,
                                        attention_mask=attention_mask,
                                        max_new_tokens=512,
                                        pad_token_id=tokenizer.eos_token_id)

    # Decode the generated tokens to get the response
    generated_tokens = response_token_ids[:, len(input_ids[0]):]
    generated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return generated_text



def summarize_news():
    for url in saved_news:
        content = fetch_news_content(url)
        summary = call_nvidia_model(content)
        summaries.append(summary)
    print("News articles summarized successfully.")
    return summaries

# Example usage
summaries = summarize_news()
for i, summary in enumerate(summaries):
    print(f"Summary {i+1}:\n{summary}\n")


In [ ]:
def extract_spatiotemporal_data():
    for summary in summaries:
        # Simulate temporal extraction (use current date)
        temp_date = datetime.now().strftime('%Y-%m-%d')
        temporal_data.append(temp_date)

        # Use GeoText to extract place names
        places = GeoText(summary)
        spatial_data.append(places.cities)

    print("Spatiotemporal data extracted.")
    return temporal_data, spatial_data

# Example usage
temporal_data, spatial_data = extract_spatiotemporal_data()
print(f"Temporal Data: {temporal_data}")
print(f"Spatial Data: {spatial_data}")


In [ ]:
def lda_topic_modeling():
    texts = [summary.split() for summary in summaries]  # Tokenize summaries
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda_model = gensim.models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=10)
    topics = lda_model.print_topics()

    print("LDA Topic Modeling Complete.")
    for topic in topics:
        print(topic)
    return topics

# Example usage
topics = lda_topic_modeling()


In [ ]:
def plot_spatiotemporal_data():
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    fig, ax = plt.subplots(1, 1)
    world.boundary.plot(ax=ax)

    # Simulate plotting locations
    for i, cities in enumerate(spatial_data):
        for city in cities:
            print(f"Plotting {city} from article {i+1}")
            # In reality, you would geocode city names to plot on the map

    plt.title("Spatiotemporal Data (Locations of Alzheimer's News)")
    plt.show()

def plot_lda_topics():
    plt.figure(figsize=(8, 4))
    plt.bar(["Topic 1", "Topic 2", "Topic 3"], [5, 10, 7])  # Simulate topic counts
    plt.title("LDA Topic Distribution")
    plt.show()

# Example usage
plot_spatiotemporal_data()
plot_lda_topics()
